# Project II: Economic Growth 

This notebook will help you getting started with analyzing the growth dataset, `growth.csv`.

In [1]:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures
import numpy.linalg as la

## Read data 

In [2]:
dat = pd.read_csv('growth.csv')
lbldf = pd.read_csv('labels.csv', index_col='variable')
lbl_all = lbldf.label.to_dict() # as a dictionary
print(f'The data contains {dat.shape[0]} rows (countries) and {dat.shape[1]} columns (variables).')

The data contains 214 rows (countries) and 85 columns (variables).


# Descriptive plots

In [3]:
lbldf.loc['gdp_growth']

label     Annual growth in GDP per capita, 1970-2020
source                                            WB
Name: gdp_growth, dtype: object

# Adding more controls

In [4]:
# all available variables
vv_institutions = ['marketref', 'dem', 'demCGV', 'demBMR', 'demreg'] 
vv_geography = [
        'tropicar','distr', 'distcr', 'distc','suitavg','temp', 'suitgini', 'elevavg', 'elevstd',
        'kgatr', 'precip', 'area', 'abslat', 'cenlong', 'area_ar', 'rough','landlock', 
        'africa',  'asia', 'oceania', 'americas' # 'europe' is the reference
]
vv_geneticdiversity = ['pdiv', 'pdiv_aa', 'pdivhmi', 'pdivhmi_aa']
vv_historical = ['pd1000', 'pd1500', 'pop1000', 'pop1500', 'ln_yst'] # these are often missing: ['pd1', 'pop1']
vv_religion = ['pprotest', 'pcatholic', 'pmuslim']
vv_danger = ['yellow', 'malfal',  'uvdamage']
vv_resources = ['oilres', 'goldm', 'iron', 'silv', 'zinc']
vv_educ = ['ls_bl', 'lh_bl'] # secondary, tertiary: we exclude 'lp_bl' (primary) to avoid rank failure 

vv_all = {'institutions': vv_institutions, 
          'geography': vv_geography, 
          'geneticdiversity': vv_geneticdiversity,
          'historical': vv_historical,
          'religion': vv_religion,
          'danger':vv_danger, 
          'resources':vv_resources
         }
list_of_lists = vv_all.values()
vv_all['all'] = [v for sublist in list_of_lists for v in sublist]

In [5]:
vs = vv_all['geography'] + vv_all['religion']
xs = ['lgdp_initial', 'pop_growth', 'investment_rate'] + vs

# avoiding missings
all_vars = ['gdp_growth'] + xs
I = dat[all_vars].notnull().all(1)

# extract data
X = dat.loc[I, xs].values
y = dat.loc[I,'gdp_growth'].values.reshape((-1,1)) * 100. #easier to read output when growth is in 100%

# Convert NaN values in X to zero
X = np.nan_to_num(X, nan=0.0)

# add const. (unless this breaks the rank condition)
oo = np.ones((I.sum(),1))
X = np.hstack([X, oo])
xs.append('constant') # we put it in as the last element

# check the rank condition
K = X.shape[1]
assert np.linalg.matrix_rank(X) == X.shape[1], f'X does not have full rank'

# compute the OLS estimator
betas = np.linalg.inv(X.T @ X) @ X.T @ y

In [6]:
# format nicely
print(f'Mean y = {y.mean(): 5.2f}% growth per year')
pd.DataFrame({'β': betas[:,0]}, index=xs).round(3)

Mean y =  1.52% growth per year


,β
lgdp_initial,-1.118
pop_growth,6.830
investment_rate,0.068
tropicar,-0.411
distr,-0.000
distcr,0.001
distc,-0.003
suitavg,-1.063
temp,-0.204
suitgini,-0.771


In [7]:


# Load data
dat = pd.read_csv('growth.csv')

# --- Variables ---
y_col = 'gdp_growth'       # Outcome
d_col = 'lgdp_initial'     # Treatment

# --- Controls (exactly as specified) ---
controls = [
    # Institutions
    'marketref', 'dem', 'demCGV', 'demreg',
    # Geography
    'tropicar', 'distr', 'distcr', 'suitavg', 'temp', 'suitgini', 'elevavg', 'elevstd',
    'kgatr', 'precip', 'area', 'abslat', 'cenlong', 'area_ar', 'rough', 'landlock',
    'africa', 'asia', 'oceania', 'americas',
    # danger
    'yellow', 'malfal',  'uvdamage',
    # Miscellaneous
    'investment_rate', 'pop_growth'
]

# Keep relevant columns and drop rows with missing data
cols_needed = [y_col, d_col] + controls
dat_sub = dat[cols_needed].dropna()

y = dat_sub[[y_col]].values*100
d = dat_sub[[d_col]].values
Z = dat_sub[controls].values

# Add polynomial features
# Hint: remember, you don't want the constant
Z_poly = PolynomialFeatures(2, include_bias=False).fit_transform(Z)

# Display number of regressors
print("The number of regressors in Z is {}".format(Z_poly.shape[1]))

N = y.shape[0]
K = Z_poly.shape[1]
print(f"N = {N}, K_controls = {K}")


The number of regressors in Z is 464
N = 76, K_controls = 464


In [8]:
def standardize(X):
    X = np.asarray(X, dtype=float)
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, ddof=1, keepdims=True)
    sd[sd == 0] = 1.0
    return (X - mu) / sd

def BRT(X_tilde, y):
    X_tilde = np.asarray(X_tilde, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1)
    N, p = X_tilde.shape
    sigma = np.std(y, ddof=1)
    c = 1.1
    alpha = 0.05
    return float((sigma * c) / np.sqrt(N) * norm.ppf(1 - alpha / (2 * p)))

def BCCH(X_tilde, y, c=1.1, alpha=0.05, max_iter=10000):
    X_tilde = np.asarray(X_tilde, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1)
    N, p = X_tilde.shape

    if p == 0:
        raise ValueError('Design matrix must have at least one column for BCCH penalty.')

    quant = norm.ppf(1 - alpha / (2 * p))

    y_centered = y - y.mean()
    yXscale = np.sqrt(np.max((X_tilde ** 2).T @ (y_centered ** 2) / N))
    penalty_pilot = c * quant * yXscale / np.sqrt(N)

    fit_pilot = Lasso(alpha=penalty_pilot, max_iter=max_iter).fit(X_tilde, y)
    eps = y - fit_pilot.predict(X_tilde)

    epsXscale = np.sqrt(np.max((X_tilde ** 2).T @ (eps ** 2) / N))
    penalty_bcch = c * quant * epsXscale / np.sqrt(N)
    return float(penalty_bcch)


In [9]:
X = np.column_stack([d, ])
xx = np.column_stack([np.ones(N), X])
coefs_OLS = la.inv(xx.T @ xx) @ (xx.T @ y)
alpha_OLS = float(coefs_OLS[1,0])

res = y - xx @ coefs_OLS
sigma2 = float((res.T @ res) / (N - xx.shape[1]))
var = sigma2 * la.inv(xx.T @ xx)
se_OLS = float(np.sqrt(var[1,1]))

t_OLS = alpha_OLS / se_OLS
print(f"OLS: alpha = {alpha_OLS:.4f}, se = {se_OLS:.4f}, t = {t_OLS:.2f}")

OLS: alpha = -0.1324, se = 0.1189, t = -1.11


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/2546114099.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2 = float((res.T @ res) / (N - xx.shape[1]))


In [10]:
# BCCH, 29
# Standardize data
Z_stan = standardize(Z)
X_stan = standardize(np.column_stack([d, Z]))

# Step 1: Lasso y on [d, Z]
lam_yx = BCCH(X_stan, y)
fit_yx = Lasso(alpha=lam_yx, max_iter=10000).fit(X_stan, y.ravel())
coefs_yx = fit_yx.coef_
sel_y = (coefs_yx[1:] != 0)  # skip the first (treatment)

# Step 2: Lasso d on Z
lam_dz = BCCH(Z_stan, d)
fit_dz = Lasso(alpha=lam_dz, max_iter=10000).fit(Z_stan, d.ravel())
sel_d = (fit_dz.coef_ != 0)

# Step 3: Union of selected controls
S = np.where(sel_y | sel_d)[0]
print(f"|S_y|={sel_y.sum()}, |S_d|={sel_d.sum()}, |S|={S.size}")

# Step 4: Post-OLS on d + selected controls
Z_S = Z[:, S] if S.size > 0 else np.empty((N, 0))
X_pdl = np.column_stack([np.ones(N), d, Z_S])
beta_pdl = la.inv(X_pdl.T @ X_pdl) @ (X_pdl.T @ y)
alpha_PDL = float(beta_pdl[1, 0])

res_pdl = y - X_pdl @ beta_pdl
sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))
var_pdl = sigma2_pdl * la.inv(X_pdl.T @ X_pdl)
se_PDL = float(np.sqrt(var_pdl[1, 1]))

t_PDL = alpha_PDL / se_PDL
print(f"PDL: alpha = {alpha_PDL:.4f}, se = {se_PDL:.4f}, t = {t_PDL:.2f}")
print("Selected controls:", [controls[i] for i in S])

# Calculate the quantile of the standard normal distribution that corresponds to the 95% confidence interval of a two-sided test
q = norm.ppf(1-0.025)

# Calculate confidence interval
CI_low_PDL  = alpha_PDL - q * se_PDL
CI_high_PDL = alpha_PDL + q * se_PDL

# Display confidence interval
print("CI_PDL = ",(CI_low_PDL.round(5),CI_high_PDL.round(5)))
print(f"lambda_yx={lam_yx:.2f}")
print(f"lambda_dz={lam_dz:.2f}")

|S_y|=0, |S_d|=1, |S|=1
PDL: alpha = -0.6854, se = 0.1514, t = -4.53
Selected controls: ['uvdamage']
CI_PDL =  (-0.98206, -0.38876)
lambda_yx=1.18
lambda_dz=0.91


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/1132448445.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))


In [11]:
# BCCH, 434
# Standardize data
Z_stan = standardize(Z_poly)
# Use Z_poly (polynomial features) together with d for the X matrix
X_stan = standardize(np.column_stack([d, Z_poly]))

# Step 1: Lasso y on [d, Z]
lam_yx = BCCH(X_stan, y)
fit_yx = Lasso(alpha=lam_yx, max_iter=10000).fit(X_stan, y.ravel())
coefs_yx = fit_yx.coef_
sel_y = (coefs_yx[1:] != 0)  # skip the first (treatment)

# Step 2: Lasso d on Z
lam_dz = BCCH(Z_stan, d)
fit_dz = Lasso(alpha=lam_dz, max_iter=10000).fit(Z_stan, d.ravel())
sel_d = (fit_dz.coef_ != 0)

# Step 3: Union of selected controls
S = np.where(sel_y | sel_d)[0]
print(f"|S_y|={sel_y.sum()}, |S_d|={sel_d.sum()}, |S|={S.size}")

# Step 4: Post-OLS on d + selected controls
Z_S = Z_poly[:, S] if S.size > 0 else np.empty((N, 0))
X_pdl = np.column_stack([np.ones(N), d, Z_S])
beta_pdl = la.inv(X_pdl.T @ X_pdl) @ (X_pdl.T @ y)
alpha_PDL = float(beta_pdl[1, 0])

res_pdl = y - X_pdl @ beta_pdl
sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))
var_pdl = sigma2_pdl * la.inv(X_pdl.T @ X_pdl)
se_PDL = float(np.sqrt(var_pdl[1, 1]))

try:
    poly = PolynomialFeatures(2, include_bias=False)
    poly.fit(Z)
    feature_names = poly.get_feature_names_out(controls)
    selected_names = [feature_names[i] for i in S]
except Exception:
    selected_names = S.tolist()
t_PDL = alpha_PDL / se_PDL
print(f"PDL: alpha = {alpha_PDL:.4f}, se = {se_PDL:.4f}, t = {t_PDL:.2f}")
print("Selected polynomial controls:", selected_names)

# Calculate the quantile of the standard normal distribution that corresponds to the 95% confidence interval of a two-sided test
q = norm.ppf(1-0.025)

# Calculate confidence interval
CI_low_PDL  = alpha_PDL - q * se_PDL
CI_high_PDL = alpha_PDL + q * se_PDL

# Display confidence interval
print("CI_PDL = ",(CI_low_PDL.round(5),CI_high_PDL.round(5)))
print(f"lambda_yx={lam_yx:.2f}")
print(f"lambda_dz={lam_dz:.2f}")


|S_y|=0, |S_d|=0, |S|=0
PDL: alpha = -0.1324, se = 0.1189, t = -1.11
Selected polynomial controls: []
CI_PDL =  (-0.3655, 0.1006)
lambda_yx=2.61
lambda_dz=1.12


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/315919839.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))


### Note we also created BRT but does not use them

In [12]:
# BRT, 29 
# Standardize data
Z_stan = standardize(Z)
X_stan = standardize(np.column_stack([d, Z]))

# Step 1: Lasso y on [d, Z]
lam_yx = BRT(X_stan, y)
fit_yx = Lasso(alpha=lam_yx, max_iter=10000).fit(X_stan, y.ravel())
coefs_yx = fit_yx.coef_
sel_y = (coefs_yx[1:] != 0)  # skip the first (treatment)

# Step 2: Lasso d on Z
lam_dz = BRT(Z_stan, d)
fit_dz = Lasso(alpha=lam_dz, max_iter=10000).fit(Z_stan, d.ravel())
sel_d = (fit_dz.coef_ != 0)

# Step 3: Union of selected controls
S = np.where(sel_y | sel_d)[0]
print(f"|S_y|={sel_y.sum()}, |S_d|={sel_d.sum()}, |S|={S.size}")

# Step 4: Post-OLS on d + selected controls
Z_S = Z[:, S] if S.size > 0 else np.empty((N, 0))
X_pdl = np.column_stack([np.ones(N), d, Z_S])
beta_pdl = la.inv(X_pdl.T @ X_pdl) @ (X_pdl.T @ y)
alpha_PDL = float(beta_pdl[1, 0])

res_pdl = y - X_pdl @ beta_pdl
sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))
var_pdl = sigma2_pdl * la.inv(X_pdl.T @ X_pdl)
se_PDL = float(np.sqrt(var_pdl[1, 1]))

t_PDL = alpha_PDL / se_PDL
print(f"PDL: alpha = {alpha_PDL:.4f}, se = {se_PDL:.4f}, t = {t_PDL:.2f}")
print("Selected controls:", [controls[i] for i in S])

# Calculate the quantile of the standard normal distribution that corresponds to the 95% confidence interval of a two-sided test
q = norm.ppf(1-0.025)

# Calculate confidence interval
CI_low_PDL  = alpha_PDL - q * se_PDL
CI_high_PDL = alpha_PDL + q * se_PDL

# Display confidence interval
print("CI_PDL = ",(CI_low_PDL.round(5),CI_high_PDL.round(5)))
print(f"lambda_yx={lam_yx:.2f}")
print(f"lambda_dz={lam_dz:.2f}")

|S_y|=2, |S_d|=2, |S|=4
PDL: alpha = -0.8262, se = 0.1450, t = -5.70
Selected controls: ['africa', 'asia', 'malfal', 'uvdamage']
CI_PDL =  (-1.11042, -0.54198)
lambda_yx=0.57
lambda_dz=0.55


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/799901511.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))


In [13]:
# BRT, 434 
# Standardize data
Z_stan = standardize(Z_poly)
X_stan = standardize(np.column_stack([d, Z_poly]))

# Step 1: Lasso y on [d, Z]
lam_yx = BRT(X_stan, y)
fit_yx = Lasso(alpha=lam_yx, max_iter=10000).fit(X_stan, y.ravel())
coefs_yx = fit_yx.coef_
sel_y = (coefs_yx[1:] != 0)  # skip the first (treatment)

# Step 2: Lasso d on Z
lam_dz = BRT(Z_stan, d)
fit_dz = Lasso(alpha=lam_dz, max_iter=10000).fit(Z_stan, d.ravel())
sel_d = (fit_dz.coef_ != 0)

# Step 3: Union of selected controls (these are indices into Z_poly)
S = np.where(sel_y | sel_d)[0]
print(f"|S_y|={sel_y.sum()}, |S_d|={sel_d.sum()}, |S|={S.size}")

# Step 4: Post-OLS on d + selected controls
# IMPORTANT: when using Z_poly, use Z_poly[:, S] (not Z) because S indexes the polynomial-feature columns
Z_S = Z_poly[:, S] if S.size > 0 else np.empty((N, 0))
X_pdl = np.column_stack([np.ones(N), d, Z_S])
beta_pdl = la.inv(X_pdl.T @ X_pdl) @ (X_pdl.T @ y)
alpha_PDL = float(beta_pdl[1, 0])

res_pdl = y - X_pdl @ beta_pdl
sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))
var_pdl = sigma2_pdl * la.inv(X_pdl.T @ X_pdl)
se_PDL = float(np.sqrt(var_pdl[1, 1]))


try:
    poly = PolynomialFeatures(2, include_bias=False)
    poly.fit(Z)
    feature_names = poly.get_feature_names_out(controls)
    selected_names = [feature_names[i] for i in S]
except Exception:
    selected_names = S.tolist()
t_PDL = alpha_PDL / se_PDL
print(f"PDL: alpha = {alpha_PDL:.4f}, se = {se_PDL:.4f}, t = {t_PDL:.2f}")
print("Selected polynomial controls:", selected_names)

# Calculate the quantile of the standard normal distribution that corresponds to the 95% confidence interval of a two-sided test
q = norm.ppf(1-0.025)

# Calculate confidence interval
CI_low_PDL  = alpha_PDL - q * se_PDL
CI_high_PDL = alpha_PDL + q * se_PDL

# Display confidence interval
print("CI_PDL = ",(CI_low_PDL.round(5),CI_high_PDL.round(5)))
print(f"lambda_yx={lam_yx:.2f}")
print(f"lambda_dz={lam_dz:.2f}")

|S_y|=1, |S_d|=3, |S|=4
PDL: alpha = -0.3226, se = 0.1456, t = -2.22
Selected polynomial controls: ['uvdamage', 'marketref abslat', 'demreg abslat', 'cenlong asia']
CI_PDL =  (-0.60806, -0.03715)
lambda_yx=0.71
lambda_dz=0.68


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/2618434632.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_pdl = float((res_pdl.T @ res_pdl) / (N - X_pdl.shape[1]))


In [14]:
pd.DataFrame({
    'alpha': [alpha_OLS, alpha_PDL, ],
    'se':    [se_OLS,    se_PDL,    ]
}, index=['OLS', 'PDL']).round(4)


,alpha,se
OLS,-0.1324,0.1189
PDL,-0.3226,0.1456


In [15]:
# Restrict to rows with complete data for outcome, treatment, and controls
cols_needed = ['gdp_growth', 'lgdp_initial'] + controls
data_mask = dat[cols_needed].notnull().all(axis=1)

Y_matched = dat.loc[data_mask, 'gdp_growth'].to_numpy().reshape(-1, 1)
D_matched = dat.loc[data_mask, 'lgdp_initial'].to_numpy().reshape(-1, 1)
Z_matched = dat.loc[data_mask, controls].to_numpy()
N_matched = Y_matched.shape[0]

# --- OLS without controls (restricted) ---
X_restricted = np.column_stack([np.ones(N_matched), D_matched])
beta_restricted = la.inv(X_restricted.T @ X_restricted) @ (X_restricted.T @ Y_matched)
alpha_restricted = float(beta_restricted[1, 0])

res_restricted = Y_matched - X_restricted @ beta_restricted
sigma2_restricted = float((res_restricted.T @ res_restricted) / (N_matched - X_restricted.shape[1]))
var_restricted = sigma2_restricted * la.inv(X_restricted.T @ X_restricted)
se_restricted = float(np.sqrt(var_restricted[1, 1]))
t_restricted = alpha_restricted / se_restricted

print('OLS (no controls, matched sample):')
print(f'  alpha = {alpha_restricted:.4f}')
print(f'  SE(alpha) = {se_restricted:.4f}')
print(f'  t-statistic = {t_restricted:.2f}')

# --- OLS with controls on the same sample (unrestricted) ---
X_unrestricted = np.column_stack([np.ones(N_matched), D_matched, Z_matched])
beta_unrestricted = la.inv(X_unrestricted.T @ X_unrestricted) @ (X_unrestricted.T @ Y_matched)
alpha_unrestricted = float(beta_unrestricted[1, 0])

res_unrestricted = Y_matched - X_unrestricted @ beta_unrestricted
sigma2_unrestricted = float((res_unrestricted.T @ res_unrestricted) / (N_matched - X_unrestricted.shape[1]))
var_unrestricted = sigma2_unrestricted * la.inv(X_unrestricted.T @ X_unrestricted)
se_unrestricted = float(np.sqrt(var_unrestricted[1, 1]))
t_unrestricted = alpha_unrestricted / se_unrestricted

print('OLS (with controls, matched sample):')
print(f'  alpha = {alpha_unrestricted:.4f}')
print(f'  SE(alpha) = {se_unrestricted:.4f}')
print(f'  t-statistic = {t_unrestricted:.2f}')


OLS (no controls, matched sample):
  alpha = -0.0013
  SE(alpha) = 0.0012
  t-statistic = -1.11


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/984655785.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_restricted = float((res_restricted.T @ res_restricted) / (N_matched - X_restricted.shape[1]))


OLS (with controls, matched sample):
  alpha = -0.0117
  SE(alpha) = 0.0016
  t-statistic = -7.21


/var/folders/s4/t_g3d19d071dl_561zffrd9c0000gn/T/ipykernel_50478/984655785.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma2_unrestricted = float((res_unrestricted.T @ res_unrestricted) / (N_matched - X_unrestricted.shape[1]))
